In [1]:
%reset -f

In [2]:
from sssom.io import parse_sssom_table
import pandas as pd

fn = "demo.tsv"

### Annotate

In [3]:
!runoak -i sqlite:obo:mondo annotate "Marfan syndrome" --matches-whole-text

object_id: MONDO:0007947
object_label: Marfan syndrome
matches_whole_text: true
subject_start: 1
subject_end: 15
subject_label: Marfan syndrome

---
object_id: MONDO:0007947
object_label: Marfan syndrome
matches_whole_text: true
subject_start: 1
subject_end: 15
subject_label: Marfan syndrome


### Lexmatch

In [ ]:
%%time
!runoak -i sqlite:obo:mondo lexmatch i^MONDO: i^DOID: -o {fn}
# !head demo.tsv
msdf = parse_sssom_table(fn)
msdf.df.head()

### Relationships

In [ ]:
!runoak -i sqlite:obo:mondo relationships  "Marfan syndrome" -o {fn}
# !head demo.tsv
df = pd.read_csv(fn, sep="\t")
df.head()

In [ ]:
# RO:0004003 => "has material basis in germline mutation in"
!runoak -i sqlite:obo:mondo relationships  --predicates RO:0004003 -o {fn}
# !head demo.tsv
df = pd.read_csv(fn, sep="\t")
df.head()